In [60]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
import json
from nltk.stem import PorterStemmer
from sklearn import linear_model
import nltk
from nltk.corpus import stopwords


In [57]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bensonjian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [3]:
users_reviews = []
for l in parse("review-Hawaii_10.json.gz"):
  users_reviews.append(l)

In [4]:
users_reviews[0]

{'user_id': '113965417079576625433',
 'name': 'manuel grimaldo',
 'time': 1591839903487,
 'rating': 5,
 'text': 'Great new upgrade',
 'pics': None,
 'resp': None,
 'gmap_id': '0x7c00159b5b1b1d25:0x8d2d85d4a758290e'}

In [5]:
businesses = []
for l in parse("meta-Hawaii.json.gz"):
  businesses.append(l)

In [6]:
businesses[0]

{'name': 'Hale Pops',
 'address': 'Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762',
 'gmap_id': '0x7c00456eecad3111:0x8217f9600c51f33',
 'description': None,
 'latitude': 21.637795699999998,
 'longitude': -157.9207142,
 'category': ['Restaurant'],
 'avg_rating': 4.4,
 'num_of_reviews': 18,
 'price': None,
 'hours': [['Thursday', '11AM–8PM'],
  ['Friday', '11AM–8PM'],
  ['Saturday', '11AM–8PM'],
  ['Sunday', 'Closed'],
  ['Monday', '11AM–8PM'],
  ['Tuesday', '11AM–8PM'],
  ['Wednesday', '11AM–8PM']],
 'MISC': {'Service options': ['Outdoor seating', 'Takeout', 'Delivery'],
  'Popular for': ['Lunch', 'Solo dining'],
  'Accessibility': ['Wheelchair accessible entrance'],
  'Offerings': ['Comfort food', 'Quick bite'],
  'Amenities': ['Good for kids'],
  'Atmosphere': ['Casual'],
  'Crowd': ['Groups', 'Tourists'],
  'Payments': ['NFC mobile payments']},
 'state': 'Closed ⋅ Opens 11AM',
 'relative_results': ['0x7c00451360f80cf1:0x930291a38bab3132',
  '0x7c00457322571d57:0xe3974e89bbfc41ce',

## Linear Regression

In [8]:
#user avg rating
user_ratings = {}

for review in users_reviews:
    user_id = review.get('user_id')
    rating = review.get('rating')

    if user_id is not None and rating is not None:
        if user_id not in user_ratings:
            user_ratings[user_id] = {'total_rating': 0, 'count': 0}
        
        user_ratings[user_id]['total_rating'] += rating
        user_ratings[user_id]['count'] += 1

user_avg_ratings = {user_id: data['total_rating'] / data['count'] 
                    for user_id, data in user_ratings.items() if data['count'] > 0}

for user_id in list(user_avg_ratings.keys())[:5]:
    print(f"User ID: {user_id}, Average Rating: {user_avg_ratings[user_id]:.2f}")

User ID: 113965417079576625433, Average Rating: 4.91
User ID: 116655819137293331166, Average Rating: 4.73
User ID: 100834119994550070853, Average Rating: 4.94
User ID: 103207214144482097315, Average Rating: 4.64
User ID: 108526171163172578599, Average Rating: 3.48


In [9]:
#businuess img count
image_counts = {}

for review in users_reviews:
    gmap_id = review.get('gmap_id')
    images = review.get('pics')

    if gmap_id and images:
        num_images = len(images)
        if gmap_id in image_counts:
            image_counts[gmap_id] += num_images
        else:
            image_counts[gmap_id] = num_images

In [10]:
response_counts = {}

for review in users_reviews:
    gmap_id = review.get('gmap_id')
    response = review.get('resp')

    if gmap_id and response:
        if gmap_id in response_counts:
            response_counts[gmap_id] += 1
        else:
            response_counts[gmap_id] = 1

In [11]:
users_data = [{'user_id': d['user_id'], 'gmap_id': d['gmap_id'], 'rating': d['rating'],'text': d['text']} 
              for d in users_reviews if 'user_id' in d and 'gmap_id' in d and 'rating' in d and 'text' in d]

business_dict = {d['gmap_id']: {'avg_rating': d.get('avg_rating', 0), 
                                'num_of_reviews': d.get('num_of_reviews', 0)}
                 for d in businesses if 'gmap_id' in d}

dataset = []
for user_review in users_data:
    gmap_id = user_review['gmap_id']
    if gmap_id in business_dict:
        merged_data = {**user_review, **business_dict[gmap_id]}
        dataset.append(merged_data)

In [12]:
#add features
for entry in dataset:
    gmap_id = entry.get('gmap_id')
    user_id = entry.get('user_id')
    entry['user_avg_rating'] = user_avg_ratings[user_id]
    entry['response_count'] = response_counts.get(gmap_id,0)
    entry['image_counts'] = image_counts.get(gmap_id, 0)
    

In [13]:
dataset[0]

{'user_id': '113965417079576625433',
 'gmap_id': '0x7c00159b5b1b1d25:0x8d2d85d4a758290e',
 'rating': 5,
 'text': 'Great new upgrade',
 'avg_rating': 4.1,
 'num_of_reviews': 18,
 'user_avg_rating': 4.909090909090909,
 'response_count': 0,
 'image_counts': 0}

In [14]:
def MSE(predictions, labels):
    differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
    return sum(differences) / len(differences)

In [71]:
def MAE(predictions, labels):
    differences = [abs(x - y) for x, y in zip(predictions, labels)]
    return sum(differences) / len(differences)

In [15]:
# split dataset
train_data = dataset[:int(len(dataset) * 0.9)]
test_data = dataset[int(len(dataset) * 0.9):]
train_users_reviews = users_reviews[:int(len(users_reviews) * 0.9)]
test_users_reviews = users_reviews[int(len(users_reviews) * 0.9):]

### Baseline Model 1: avg_rating

In [16]:
# extract features
train_X = [[1, d['avg_rating']] for d in train_data]
test_X = [[1, d['avg_rating']] for d in test_data]
train_y = [u['rating'] for u in train_users_reviews]
test_y = [u['rating'] for u in test_users_reviews]

In [17]:
# train model
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(train_X, train_y)
predictions = model.predict(test_X)

In [18]:
mse = MSE(predictions, test_y)
mse

0.6738557818215304

### Baseline Model 2: user_avg_rating

In [19]:
train_X = [[1, d['user_avg_rating']] for d in train_data]
test_X = [[1, d['user_avg_rating']] for d in test_data]
train_y = [u['rating'] for u in train_users_reviews]
test_y = [u['rating'] for u in test_users_reviews]

In [20]:
# train model
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(train_X, train_y)
predictions = model.predict(test_X)

In [21]:
mse = MSE(predictions, test_y)
mse

0.5256570587970208

### Model 3: avg_rating, user_avg_rating

In [40]:
train_X = [[1, d['user_avg_rating'], d['avg_rating']] for d in train_data]
test_X = [[1, d['user_avg_rating'], d['avg_rating']] for d in test_data]
train_y = [u['rating'] for u in train_users_reviews]
test_y = [u['rating'] for u in test_users_reviews]

In [41]:
# train model
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(train_X, train_y)
predictions = model.predict(test_X)

In [42]:
mse = MSE(predictions, test_y)
mse

0.4886740944678856

### Model 4: avg_rating, user_avg_rating, num_of_reviews

In [25]:
train_X = [[1, d['user_avg_rating'], d['avg_rating'], d['num_of_reviews']] for d in train_data]
test_X = [[1, d['user_avg_rating'], d['avg_rating'], d['num_of_reviews']] for d in test_data]
train_y = [u['rating'] for u in train_users_reviews]
test_y = [u['rating'] for u in test_users_reviews]

In [26]:
# train model
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(train_X, train_y)
predictions = model.predict(test_X)

In [27]:
mse = MSE(predictions, test_y)
mse

0.4894779561626342

### Model 5

In [32]:
train_X = [[1, d['user_avg_rating'], d['avg_rating'], d['num_of_reviews'], d['image_counts']] for d in train_data]
test_X = [[1, d['user_avg_rating'], d['avg_rating'], d['num_of_reviews'], d['image_counts']] for d in test_data]
train_y = [u['rating'] for u in train_users_reviews]
test_y = [u['rating'] for u in test_users_reviews]

In [33]:
# train model
model = linear_model.LinearRegression(fit_intercept=False)
model.fit(train_X, train_y)
predictions = model.predict(test_X)

In [34]:
mse = MSE(predictions, test_y)
mse

0.4891960794205272

## Bag of Words

In [58]:
# split dataset
train_data = dataset[:int(len(dataset) * 0.9)]
test_data = dataset[int(len(dataset) * 0.9):]
train_users_reviews = users_reviews[:int(len(users_reviews) * 0.9)]
test_users_reviews = users_reviews[int(len(users_reviews) * 0.9):]

In [61]:
# lowercase, punctuation removed, no stemming
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stop_words = set(stopwords.words('english'))
# stemmer = PorterStemmer()
for d in train_data:
    if d['text']:
        r = ''.join([c for c in d['text'].lower() if not c in punctuation])
        ws = [w for w in r.split() if w not in stop_words]
        for w in ws:
            # w = stemmer.stem(w)
            wordCount[w] += 1
len(wordCount)

311649

In [62]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort(reverse=True)
words = [x[1] for x in counts[:1000]]

### Sentiment analysis

In [63]:
wordID = dict(zip(words, range(len(words))))
wordSet = set(words)

In [64]:
def feature(datum):
    feat = [0]*len(words)
    if datum['text']:
        r = ''.join([c for c in datum['text'].lower() if not c in punctuation])
        ws = [w for w in r.split() if w not in stop_words]
        for w in ws:
            if w in words:
                feat[wordID[w]] += 1
    feat.append(1) #offset
    return feat

In [65]:
train_X = [feature(d) for d in train_data]
train_y = [u['rating'] for u in train_users_reviews]
test_X = [feature(d) for d in test_data]
test_y = [u['rating'] for u in test_users_reviews]

In [66]:
# regularized regression
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(train_X, train_y)
theta = clf.coef_
train_predictions = clf.predict(train_X)
test_predictions = clf.predict(test_X)

In [67]:
train_mse = MSE(train_predictions, train_y)
test_mse = MSE(test_predictions, test_y)
print(f'train mse: {train_mse}')
print(f'test mse: {test_mse}')

train mse: 0.7099192239329473
test mse: 0.6224833976559787


In [72]:
train_mae = MAE(train_predictions, train_y)
test_mae = MAE(test_predictions, test_y)
print(f'train mae: {train_mae}')
print(f'test mae: {test_mae}')

train mae: 0.6629048356631284
test mae: 0.6388418997362427


In [68]:
wordSort = list(zip(theta, words))
wordSort.sort()

In [69]:
wordSort[:50]

[(-1.327276586668271, 'worst'),
 (-1.1900794552149985, 'horrible'),
 (-1.1091567561356912, 'terrible'),
 (-1.105245298123043, 'rude'),
 (-1.084665176556934, 'poor'),
 (-0.9498497118404947, 'overpriced'),
 (-0.838752728986792, 'dirty'),
 (-0.7252086338199372, 'homeless'),
 (-0.7175400282410713, 'slow'),
 (-0.5421720821710071, 'ok'),
 (-0.5116224837929986, 'sad'),
 (-0.49177798928451977, 'expensive'),
 (-0.47869827880830723, 'average'),
 (-0.46923581532118736, 'okay'),
 (-0.46166266619972285, 'closed'),
 (-0.40780496865091137, 'priced'),
 (-0.40114313289268966, 'nothing'),
 (-0.4004991815248283, 'dry'),
 (-0.38043864858591114, 'bad'),
 (-0.37642970981428686, 'sorry'),
 (-0.36285863860615264, 'waited'),
 (-0.36106212436378265, 'salty'),
 (-0.3452652029334908, 'disappointed'),
 (-0.3094833250172318, 'empty'),
 (-0.3064028190804877, 'needs'),
 (-0.291613708838876, 'money'),
 (-0.29041736393532014, 'tourist'),
 (-0.28953233755898594, 'pricey'),
 (-0.28097528016852086, 'loud'),
 (-0.277822811